# Mphasis HyperGraf Consumer Loan Delinquency Predictor

Sanctioning loans to consumers without credit history is a challenge for consumer finance companies. This solution predicts the probability of loan delinquency for such consumers. This solution can assist consumer finance companies in their decision-making process while assessing the risk to sanction loans. Companies can use the likelihood of loan delinquency to make targeted interventions to identify and mitigate potential loan defaults.

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy Mphasis HyperGraf Consumer Loan Delinquency Predictor using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Mphasis HyperGraf Consumer Loan Delinquency Predictor. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Mphasis HyperGraf Consumer Loan Delinquency Predictor
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/consumer-loan-delinquency-predictor-v2'

In [2]:
import pandas as pd
import numpy as np
import json
import os
import boto3
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image, display

In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='consumer-loan-delinquency-predictor'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [6]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

#### Instructions

The input file contains following fields:

   - SK_ID_CURR : ID of Loan
   - NAME_CONTRACT_TYPE : Type of loan - Cash loans or Revolving loans
   - CODE_GENDER : Gender of the consumer
   - FLAG_OWN_CAR : Car ownership. If the consumer owns a car, enter ‘Y’; else, enter ‘N’
   - AMT_INCOME_TOTAL : Annual Income of the consumer in USD
   - AMT_CREDIT : Credit amount of the loan in USD
   - OWN_CAR_AGE : Age of consumer's car in years 
   - FLAG_EMP_PHONE : Personal phone number- if consumer provided 1, else 0
   - FLAG_WORK_PHONE : Work phone number- if consumer provided 1, else 0
   - FLAG_PHONE : Home phone number- if consumer provided 1, else 0 
   - CNT_FAM_MEMBERS : Number of family members of the consumer
   - LIVE_CITY_NOT_WORK_CITY : Consumer location. If consumer lives and works in different cities enter 1, else 0
   - DATE_APPL : Date of loan application in MM/DD/YYYY format
   - DOB : Date of Birth of consumer in MM/DD/YYYY format
   - DATE_PHONECHANGE : Date when consumer changed phone in MM/DD/YYYY format
   - DATE_EMP : Date when consumer started current employment in MM/DD/YYYY format 
    

In [7]:
file_name = 'sample_input.csv'

In [18]:
input_data = pd.read_csv(file_name)
input_data

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,CNT_FAM_MEMBERS,LIVE_CITY_NOT_WORK_CITY,DATE_APPL,DOB,DATE_PHONECHANGE,DATE_EMP
0,1,Cash loans,F,N,157500,723996,0,1,0,1,2,1,7/17/2021,9/21/1992,5/22/2021,10/23/2020
1,2,Cash loans,M,N,292500,675000,0,1,1,1,2,0,2/1/2021,6/16/1975,10/18/2020,7/16/2020
2,3,Cash loans,F,N,157500,245619,0,1,0,1,1,0,3/13/2021,3/7/1973,8/9/2020,3/7/2000
3,5,Revolving loans,M,N,180000,540000,0,1,0,0,5,0,7/17/2021,8/1/1979,5/14/2020,9/13/2018
4,6,Cash loans,F,N,202500,436032,0,1,0,1,3,0,3/13/2021,6/21/1993,11/3/2020,11/22/2020
5,7,Revolving loans,M,N,81000,247500,0,1,1,1,2,1,8/12/2021,8/27/1971,2/26/2021,3/6/2015
6,8,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021
7,9,Revolving loans,M,Y,225000,270000,1,1,0,0,2,0,2/1/2021,6/23/1973,5/26/2019,7/23/2018
8,11,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021
9,13,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021


#### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 output.csv

{
    "ContentType": "csv",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- The output file ('output.csv') contains all the columns of sample_input.csv along with **PRED_DELINQ**
- PRED_DELINQ : Risk of loan delinquency- 1 indicates high risk of delinquent payments whereas 0 indicates that the consumer is highly likely to payback on time

In [9]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df

Sample Output:


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,CNT_FAM_MEMBERS,LIVE_CITY_NOT_WORK_CITY,DATE_APPL,DOB,DATE_PHONECHANGE,DATE_EMP,PRED_DELINQ
0,1,Cash loans,F,N,157500,723996,0,1,0,1,2,1,7/17/2021,9/21/1992,5/22/2021,10/23/2020,1
1,2,Cash loans,M,N,292500,675000,0,1,1,1,2,0,2/1/2021,6/16/1975,10/18/2020,7/16/2020,1
2,3,Cash loans,F,N,157500,245619,0,1,0,1,1,0,3/13/2021,3/7/1973,8/9/2020,3/7/2000,1
3,5,Revolving loans,M,N,180000,540000,0,1,0,0,5,0,7/17/2021,8/1/1979,5/14/2020,9/13/2018,1
4,6,Cash loans,F,N,202500,436032,0,1,0,1,3,0,3/13/2021,6/21/1993,11/3/2020,11/22/2020,1
5,7,Revolving loans,M,N,81000,247500,0,1,1,1,2,1,8/12/2021,8/27/1971,2/26/2021,3/6/2015,0
6,8,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021,0
7,9,Revolving loans,M,Y,225000,270000,1,1,0,0,2,0,2/1/2021,6/23/1973,5/26/2019,7/23/2018,0
8,11,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021,0
9,13,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021,0


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [10]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [11]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/consumer-loan-delinquency-predictor


In [12]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

....................... * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
 * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [13/Jul/2021 11:28:00] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [13/Jul/2021 11:28:01] "#033GET /execution-parameters HTTP/1.1#033" 404 -
Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'OWN_CAR_AGE', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_PHONE', 'CNT_FAM_MEMBERS',
       'LIVE_CITY_NOT_WORK_CITY', 'DATE_APPL', 'DOB', 'DATE_PHONECHANGE',
       'DATE_EMP'],
      dtype='object')
169.254.255.130 - - [13/Jul/2021 11:28

In [13]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/consumer-loan-delinquency-predictor-v2-2021-07-13-11-24-20-605'

In [14]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output2.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/sample_input.csv.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [15]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output2.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)
output_df

Sample Output:


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,AMT_INCOME_TOTAL,AMT_CREDIT,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,CNT_FAM_MEMBERS,LIVE_CITY_NOT_WORK_CITY,DATE_APPL,DOB,DATE_PHONECHANGE,DATE_EMP,PRED_DELINQ
0,1,Cash loans,F,N,157500,723996,0,1,0,1,2,1,7/17/2021,9/21/1992,5/22/2021,10/23/2020,1
1,2,Cash loans,M,N,292500,675000,0,1,1,1,2,0,2/1/2021,6/16/1975,10/18/2020,7/16/2020,1
2,3,Cash loans,F,N,157500,245619,0,1,0,1,1,0,3/13/2021,3/7/1973,8/9/2020,3/7/2000,1
3,5,Revolving loans,M,N,180000,540000,0,1,0,0,5,0,7/17/2021,8/1/1979,5/14/2020,9/13/2018,1
4,6,Cash loans,F,N,202500,436032,0,1,0,1,3,0,3/13/2021,6/21/1993,11/3/2020,11/22/2020,1
5,7,Revolving loans,M,N,81000,247500,0,1,1,1,2,1,8/12/2021,8/27/1971,2/26/2021,3/6/2015,0
6,8,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021,0
7,9,Revolving loans,M,Y,225000,270000,1,1,0,0,2,0,2/1/2021,6/23/1973,5/26/2019,7/23/2018,0
8,11,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021,0
9,13,Revolving loans,M,N,225000,270000,0,1,0,0,1,1,8/12/2021,7/25/1979,10/20/2019,4/20/2021,0


### 4. Clean-up

#### A. Delete the model

In [16]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.